In [1]:
from Network_PER import *
from DriveSimTrace import *
import shap

pygame 2.1.2 (SDL 2.0.18, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\kyjun\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SHAP, Decomposed Reward 정보를 통한 설명 텍스트 생성

In [2]:
def pred_explain(pred_list):
    pred_label = ["reach finish line", "avoid the obstacle", "avoid the wall"]
    action_label = ["Idle", "Accel", "Decel", "Turn Left", "Turn Right"]

    pred_sum = np.sum(pred_list, axis=0)
    chosen = np.argmax(pred_sum)
    compare = np.argmin(pred_sum)

    diff = np.transpose(pred_list)[chosen] - np.transpose(pred_list)[compare]
    diff = diff[0]

    txt = [f"I chose '{action_label[chosen]}' rather than '{action_label[compare]}' because it seemed more likely to help me ",""]
    
    for i in range(np.size(diff)):
        if diff[i] > 0:
            txt[1] += f"{pred_label[i]}(+{ str(round(diff[i], 2)) })/"
    
    txt[1] = txt[1][:-1] + "."

    return txt

def shap_explain(shap_list):
    state_label = ["Speed","Rotation","Agent X Position","Agent Y Position",\
        "Obstacle X Position","Obstacle Y Position","Obstacle Size"]

    shap_list = shap_list[0,0]
    imp1 = np.argmax(shap_list)
    imp1val = shap_list[imp1]
    shap_list[imp1] = -100000

    imp2 = np.argmax(shap_list)
    imp2val = shap_list[imp2]

    txt = [f"I considered '{state_label[imp1]}'({ str(round(imp1val, 3)) }) and '{state_label[imp2]}'({ str(round(imp2val, 3)) })",\
     f"the most important in making this decision."]
    return txt

시뮬레이션 설명

In [5]:
if __name__ == '__main__':
    import silence_tensorflow
    silence_tensorflow.silence_tensorflow()

    #시뮬레이션 테스트 코드
    n_runs = 1000
    for k in range(1):
        model_path = f'./models/q_20230226(0)'

        #train_scores_1 = np.genfromtxt('20221003_concat_train_scores_1.csv', delimiter=',')

        wins = 0
        agent = Agent(lr=0, gamma=0.99, n_actions=5, batch_size=64, epsilon=0.0, input_dims=[1, 7], per_on=False)

        agent.load_model(model_path)

        sim = DriveSimulator()
        avg_score = 0.0


        collect_bg = 1
        background = []

        for i in range(n_runs):
            over = False
            sim.reset(frame_rate=3000)
            state = sim.get_sim_state()

            print(i)
            if i >= collect_bg:
                sim.explain = True
                bg = (np.array(background))
                e_0 = shap.DeepExplainer(agent.q_evals[0], np.array([bg[0]]) )
                e_1 = shap.DeepExplainer(agent.q_evals[1], np.array([bg[0]]) )
                e_2 = shap.DeepExplainer(agent.q_evals[2], np.array([bg[0]]) )

            while not over:

                if sim.explain:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN: #and event.key == pygame.K_SPACE: # 스페이스 키 입력 시에만 프레임 넘김
                            
                            #SHAP Explainer -> 현재 state에서 가장 중요한 정보 찾기
                            shap_values = (e_0.shap_values( np.array([state]) ))
                            shap_values += (e_1.shap_values( np.array([state]) ))
                            shap_values += (e_2.shap_values( np.array([state]) ))

                            #시뮬레이션 업데이트
                            action, pred, pred_C = agent.choose_action(state)
                            state_, stpRwd, sim_over = sim.step(action, pred_C)
                            over = sim_over
                            state = state_


                            #설명(SHAP+Reward Decomposition) 화면에 표시
                            sim.render_explanation(shap_values, pred_C, \
                                 shap_explain(shap_values[action]), pred_explain(pred_C))

                else:
                    background.append(np.array(state))
                
                    action, pred, pred_C = agent.choose_action(state)
                    state_, stpRwd, sim_over = sim.step(action, pred_C)
                    over = sim_over
                    state = state_
                 
            avg_score += sim.agtRwd
            if sim.agtRwd[0] > 0:
                wins+=1

        print(wins)

Use Prioritized Sampling: False
loaded weights from ./models/q_20230226(0)
0
1


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


AttributeError: 'Event' object has no attribute 'key'